In [1]:
import h5py
import matplotlib.pyplot as plt
from Asp_smth_funcs_opt import SAVGOL_smoothing, WT_smoothing, SNR_diff, plot_WT, plot_SAVGOL
from Asp_baselinne_funcs_opt import als_baseline, polym_fitting_baseline
import numpy
import scipy.io
import os
from PIL import Image
import numpy as np
from numpy import asarray
from scipy.signal import savgol_filter
from scipy.signal import detrend
import pandas as pd
from scipy.optimize import curve_fit
# from statsmodels.tsa.tsatools import detrend
from scipy import sparse
from scipy.sparse.linalg import spsolve
import pywt
from math import log10, sqrt
import time
import pysptools
from pysptools import distance
import math
import pandas as pd
import openpyxl


In [ ]:
# Getting Asprin data
# Construct the full path of the file 
file_path = os.path.join(os.getcwd(), 'Asprin.xlsx')

# Read the Excel file which has the downloaded corrrect Asprin data into a pandas DataFrame
wavelengths = pd.read_excel(file_path, usecols = 'A', header = None)
wavelengths_arr = (wavelengths.values).flatten()

intensity = pd.read_excel(file_path, usecols = 'B', header = None)
intensity_arr = (intensity.values).flatten()

fig, axs = plt.subplots(3)
fig.suptitle('Vertically stacked subplots')
axs[0].plot(wavelengths_arr, intensity_arr, lw=0.5)

In [3]:
# Function for adding noise
mu=0.0
std = 0.70 * np.std(intensity_arr)
def gaussian_noise(x,mu,std):
    noise = np.random.normal(mu, std, size = x.shape)
    x_noisy = x + noise
    return x_noisy, noise

In [ ]:
# RUNNING THE WT AND SAVGOL SMOOTHING - can change the level of noise above (done for 0.6/0.7/0.8/0.9)
true_signal_power = np.mean(np.abs(intensity_arr)**2)
noise_power = np.mean(np.abs(generated_noise)**2)

plot_WT(noisy_spec, intensity_arr, true_signal_power, noise_power)
plot_SAVGOL(noisy_spec, intensity_arr, true_signal_power, noise_power)

Checking which wl and pO is best for savgol

In [56]:
max_snr = -np.inf
optimal_window_length = None

true_signal_power = np.mean(np.abs(intensity_arr)**2)

for i in range(100): #do this process over 100 diff random noise added spectra
    noisy_spec, generated_noise = gaussian_noise(intensity_arr, mu, std)

    noise_power = np.mean(np.abs(generated_noise)**2)

    # Iterate over a range of polynomial orders
    for pO in range(1, 9):
    # Iterate over a range of window lengths
        for wl in range(9, 151, 2):
            # Skip if the polynomial order is greater than or equal to the window length
            if pO >= wl:
                continue
            smthed_noise, _ = SAVGOL_smoothing(noisy_spec, intensity_arr, wl, pO)
            
            # Calculate the SNR for this window length
            rel_snr, abs_snr = SNR_diff(true_signal_power, noise_power, smthed_noise)

            # If this SNR is greater than the current maximum...
            if rel_snr > max_snr:
                    # Update the maximum SNR and optimal window length
                max_snr = rel_snr
                optimal_pO = pO
                optimal_window_length = wl

# Print the optimal window length
print(f"The optimal wl and pO is {optimal_window_length, optimal_pO}")

# After the loop, plot the smoothed spectra
    # plt.figure(figsize=(10, 6))
    # for (poly_order, window_l), spectrum in smoothed_spectra.items():
    #     print(f'Poly order: {poly_order}, Window length: {window_l}, Spectrum: {spectrum}') # Modify this line with what being varied
    #     plt.plot(spectrum, label=f'Poly order: {poly_order}, Window length: {window_l}')
    # plt.title('Smoothed spectra for different parameters')
    # plt.xlabel('Wavelength index')
    # plt.ylabel('Intensity')
    # plt.legend()
    # plt.show()

In [ ]:
# x = wavelengths_arr
# y = -10 * x + 5 + np.random.normal(0, 1, len(x)) # arbitary linear trend added
#
# noisy_trend_spec = noisy_spec + y

In [ ]:
#verify smth should come first
percdiff_detrsmth = []
percdiff_smthdetr =[]
for i in range(5):
    noisy_spec, generated_noise = gaussian_noise(intensity_arr, mu, std)
    x = wavelengths_arr
    y = -10 * x + 5 + np.random.normal(0, 1, len(x))  # arbitary linear trend added
    noisy_trend_spec = noisy_spec + y

    #SmthDetr
    smth1 = savgol_filter(noisy_trend_spec, 11,3)
    detr1 = detrend(smth1, type = 'linear')

    #DetrSmth
    detr2 = detrend(noisy_trend_spec, type = 'linear')
    smth2 = savgol_filter(detr2, 11,3)

    perct_diff1 = 0
    for value1, value2 in zip(intensity_arr, detr1):
        if value1 == 0:
            pass
        else:
            perct_diff1 = abs((value1 - value2) / value1) * 100
            perct_diff1 += perct_diff1
    percdiff_smthdetr.append(np.mean(perct_diff1))

    perct_diff2 = 0
    for value3, value4 in zip(intensity_arr, smth2):
        if value3 == 0:
            pass
        else:
            perct_diff2 = abs((value3 - value4) / value3) * 100
            perct_diff2 += perct_diff2
    percdiff_detrsmth.append(np.mean(perct_diff2))


data_dict1 = {'trials': percdiff_detrsmth
             }

data_dict2 = {'trials': percdiff_smthdetr
             }

df1 = pd.DataFrame(data_dict1)
df1.to_csv('detrsmth.csv')

df2 = pd.DataFrame(data_dict2)
df2.to_csv('smthdetr.csv')

In [ ]:
# Adding baseline to Asprin

noisy_spectrum = noisy_spec + 10000  #all you need to do to generate a baseline

# polym_fitting_baseline(4, wavelengths_arr, noisy_spec)
perc_diff_all_asymm = []
for smooth in [10000, 100000, 1000000, 10000000, 100000000, 1000000000]:
    perc_diff_asymm = []
    for asymm in np.arange(0, 1.01, 0.05):
        baselined_spec = als_baseline(noisy_spectrum, smooth, asymm) # start at asymm = 0.5, tested both sides, more positive wasnt good, so went closer ot 0
        perct_diff = 0
        for value1, value2 in zip(noisy_spec, baselined_spec):
            if value1 == 0:
                pass
            else:
                perct_diff = abs((value1 - value2) / value1) * 100
                perct_diff += perct_diff
                #perct_diff = np.mean(np.array(abs((value1 - value2) / value1) * 100))
        perc_diff_asymm.append(np.mean(perct_diff))
    perc_diff_all_asymm.append(perc_diff_asymm)

data_dict = {'asym value': np.arange(0, 1.01, 0.05),
             '10000': perc_diff_all_asymm[0],
             '100000': perc_diff_all_asymm[1],
             '1000000': perc_diff_all_asymm[2],
             '10000000': perc_diff_all_asymm[3],
             '100000000': perc_diff_all_asymm[4],
             '1000000000': perc_diff_all_asymm[5],
             }

df = pd.DataFrame(data_dict)
df.to_csv('noise_als_baseline.csv')

print(perc_diff_all_asymm)
min_value = float('inf')  # Initialise to positive infinity
min_indices = (-1, -1, -1)  # Initialise with invalid indices

for i, list in enumerate(perc_diff_all_asymm):
    for j, value in enumerate(list):
        if value < min_value:
            min_value = value
            min_indices = (i, j)


#PRINT MIN PERCRENTAGED DIFF
print(f"Minimum value={min_value} is at location={min_indices}")
